### Basic Workflow Creation


pip install llama-index llama-index-vector-stores-chroma llama-index-utils-workflow llama-index-llms-huggingface-api pyvis

In [1]:
from llama_index.core.workflow import StartEvent, StopEvent, Workflow, step

class MyWorkFlow(Workflow):
    @step
    async def my_step(self, ev: StartEvent) -> StopEvent:
        return StopEvent(result="Hello, world!")


w = MyWorkFlow(timeout=10, verbose=False)
result = await w.run()

result

'Hello, world!'

### Connecting Multiple Steps

We can also create multi-step workflows. Here we pass the event information between steps. Note that we can use type hinting to specify the event type and the flow of the workflow.

In [3]:
from llama_index.core.workflow import Event

class ProcessingEvent(Event):
    intermediate_result: str

class MultiStepWorkflow(Workflow):
    @step
    async def step_one(self, ev: StartEvent) -> ProcessingEvent:
        # process initial data
        return ProcessingEvent(intermediate_result="Step 1 complete")
    
    @step
    async def step_two(self, ev: ProcessingEvent) -> StopEvent:
        # Use the intermediate result
        final_result = f"Finished processing: {ev.intermediate_result}"
        return StopEvent(result=final_result)
    
w = MultiStepWorkflow(timeout=10, verbose=False)
result = await w.run()
result

'Finished processing: Step 1 complete'

### Loops and Branches
We can also use type hinting to create branches and loops. Note tha we can use the `|` operator to specify that the step can return multiple types.

In [9]:
from llama_index.core.workflow import Event
import random

class ProcessingEvent(Event):
    intermediate_result: str

class LoopEvent(Event):
    loop_output: str

class MultiStepWorkflow(Workflow):
    @step
    async def step_one(self, ev: StartEvent | LoopEvent) -> ProcessingEvent | LoopEvent:
        if random.randint(0, 1) == 0:
            print("Bas thing happened")
            return LoopEvent(loop_output="Back to step one.")
        else:
            print("Good thing happened.")
            return ProcessingEvent(intermediate_result="First step complete")
        
    @step
    async def step_two(self, ev: ProcessingEvent) -> StopEvent:
        # use the intermediate result
        final_result = f"Finished processing: {ev.intermediate_result}"
        return StopEvent(result=final_result)
    
w = MultiStepWorkflow(verbose=False)
result = await w.run()
result

Bas thing happened
Bas thing happened
Bas thing happened
Bas thing happened
Good thing happened.


'Finished processing: First step complete'

### Drawing workflows

We can also draw workflows using the `draw_all_possible_flows` function.

In [10]:
from llama_index.utils.workflow import draw_all_possible_flows

draw_all_possible_flows(w)

workflow_all_flows.html


### State Management

Instead of passing the event information between steps, we can use the `Context` type hint to pass information between steps. This might be useful for long running workflows,
Where you want to store information between steps.

In [11]:
from llama_index.core.workflow import Event, Context
from llama_index.core.agent.workflow import ReActAgent

class ProcessingEvent(Event):
    intermediate_result: str

class MultiStepWorkflow(Workflow):
    @step
    async def step_one(self, ev: StartEvent, ctx: Context) -> ProcessingEvent:
        # processing initial data
        await ctx.store.set("query", "What is the capital of France?")
        return ProcessingEvent(intermediate_result="Step 1 complete")
    
    @step
    async def step_two(self, ev: ProcessingEvent, ctx: Context) -> StopEvent:
        # Use the intermediate result
        query = await ctx.store.get("query")
        print(f"Query: {query}")
        final_result = f"Finished processing: {ev.intermediate_result}"
        return StopEvent(result=final_result)
    
w = MultiStepWorkflow(timeout=10, verbose=False)
result = await w.run()
result

Query: What is the capital of France?


'Finished processing: Step 1 complete'

### Multi-Agent Workflows

We can also create multi-agent workflows. Here we define two agents, one that multiplies two integers and one that adds two integers.

In [14]:
from llama_index.core.agent.workflow import AgentWorkflow, ReActAgent
from llama_index.llms.ollama import Ollama
from llama_index.core.agent.workflow import AgentWorkflow

# Define some tools
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b

def multiply(a : int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

llm = Ollama(model="kimi-k2:1t-cloud", base_url='http://127.0.0.1:11434')

# We can pass functions directly without FunctionTool -- the fn/docstring are parsed for the name/description
multiply_agent = ReActAgent(
    name="multiply_agent",
    description="Is able to multiply two integers",
    system_prompt="A helpful assistant that can use a tool to multiply numbers.",
    tools=[multiply],
    llm=llm,
)

addition_agent = ReActAgent(
    name = "add_agent",
    description = "Is able to add two integers",
    system_prompt="A helpful assistant that can use a tool to add numbers.",
    tools = [add],
    llm=llm,
)

# create the workflow
workflow = AgentWorkflow(
    agents=[multiply_agent, addition_agent],
    root_agent="multiply_agent"
)

# Run the system
response = await workflow.run(user_msg="Can you add 5 and 3?")
response

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='5 + 3 = 8')]), structured_response=None, current_agent_name='add_agent', raw={'model': 'kimi-k2:1t-cloud', 'created_at': '2026-01-11T13:09:27.480663638Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1162937585, 'load_duration': None, 'prompt_eval_count': 822, 'prompt_eval_duration': None, 'eval_count': 30, 'eval_duration': None, 'message': Message(role='assistant', content='', thinking=None, images=None, tool_name=None, tool_calls=None), 'logprobs': None, 'usage': {'prompt_tokens': 822, 'completion_tokens': 30, 'total_tokens': 852}}, tool_calls=[ToolCallResult(tool_name='handoff', tool_kwargs={'to_agent': 'add_agent', 'reason': 'The user wants to add 5 and 3, but I only have a multiply tool. The add_agent is better suited to handle this addition request.'}, tool_id='3e519276-7156-4627-a850-507a1efc9a96', tool_output=ToolOutput(blocks=[T

Agent tools can also modify the workflow state we mentioned earlier. Before starting the workflow, we can provide an initial state dict that will be available to all agents. The state is stored in the state key of the workflow context. It will be injected into the state_prompt which augments each new user message.

Let’s inject a counter to count function calls by modifying the previous example:

In [16]:
from llama_index.core.workflow import Context

# Define some tools
async def add(ctx: Context, a: int, b: int) -> int:
    """Add two numbers."""
    # update our count
    cur_state = await ctx.store.get("state")
    cur_state["num_fn_calls"] += 1
    await ctx.store.set("state", cur_state)

    return a + b

async def multiply(ctx: Context, a: int, b: int) -> int:
    """Multiply two numbers."""
    # update our count
    cur_state = await ctx.store.get("state")
    cur_state["num_fn_calls"] += 1
    await ctx.store.set("state", cur_state)

    return a * b


# We can pass functions directly without FunctionTool -- the fn/docstring are parsed for the name/description
multiply_agent = ReActAgent(
    name="multiply_agent",
    description="Is able to multiply two integers",
    system_prompt="A helpful assistant that can use a tool to multiply numbers.",
    tools=[multiply],
    llm=llm,
)

addition_agent = ReActAgent(
    name = "add_agent",
    description = "Is able to add two integers",
    system_prompt="A helpful assistant that can use a tool to add numbers.",
    tools = [add],
    llm=llm,
)

# defining workflow
workflow = AgentWorkflow(
    agents=[multiply_agent, addition_agent],
    root_agent="multiply_agent",
    initial_state={"num_fn_calls": 0},
    state_prompt="Current state: {state}. User message: {msg}",
)

ctx = Context(workflow)
response = await workflow.run(user_msg="Can you add 5 and 3?", ctx=ctx)

# pull out and inspect the state
state = await ctx.store.get('state')
print(state["num_fn_calls"])

0
